In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import matplotlib.patheffects as patheffects

from astropy.io import fits
from astropy.wcs import WCS
from astropy.wcs import FITSFixedWarning

import os

from AstroColour.AstroColour import RGB

from ps1_table_obj import ps1_casjobs

import warnings # To ignore our problems
warnings.filterwarnings('ignore', category=FITSFixedWarning)

%matplotlib widget

In [ ]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
fig_width_pt = 244.0  # Get this from LaTeX using \the\columnwidth
text_width_pt = 508.0 # Get this from LaTeX using \the\textwidth

# plt.rc('font', size=21)
# plt.rc('xtick', labelsize=21)
# plt.rc('ytick', labelsize=21)

inches_per_pt = 1.0/72.27               # Convert pt to inches
golden_mean = (np.sqrt(5)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt*1.5 # width in inches
fig_width_full = text_width_pt*inches_per_pt  # 17
fig_height =fig_width*golden_mean # height in inches
fig_size = [fig_width,fig_height] #(9,5.5) #(9, 4.5)

In [ ]:
atclean_folder = '../../Data/'

coords = pd.read_csv(atclean_folder+'2019vxm_control_coords.txt', delim_whitespace=True)

coords

In [ ]:
fits_folder = '/Users/zgl12/Research/ATClean_FITS/'

g_fits_file = fits_folder+'rings.v3.skycell.2491.050.stk.g.unconv.fits'
r_fits_file = fits_folder+'rings.v3.skycell.2491.050.stk.r.unconv.fits'
i_fits_file = fits_folder+'rings.v3.skycell.2491.050.stk.i.unconv.fits'

hdul = fits.open(g_fits_file)
wcs = WCS(hdul[1].header)
g_data = hdul[1].data
hdul.close()

hdul = fits.open(r_fits_file)
r_data = hdul[1].data
hdul.close()

hdul = fits.open(i_fits_file)
i_data = hdul[1].data
hdul.close()

cdelt_x = wcs.wcs.cdelt[0]  # scale in RA (degrees per pixel)
cdelt_y = wcs.wcs.cdelt[1]  # scale in Dec (degrees per pixel)

cdelt_x_arcsec = cdelt_x * 3600  # arcseconds per pixel in RA
cdelt_y_arcsec = cdelt_y * 3600  # arcseconds per pixel in Dec
pixel_scale = np.nanmean([cdelt_x_arcsec, cdelt_y_arcsec])


pixel_distance = coords['radius_arcsec'].values / pixel_scale

x, y = wcs.all_world2pix(coords['ra'].values, coords['dec'].values, 0)

x_cen = 4740.9189

y_cen = 2859.1121

colour_images = []

for i in [g_data, r_data, i_data]:
    colour_images.append(i)

In [ ]:
ra_gal, dec_gal = wcs.all_pix2world(x_cen, y_cen, 0)
ra_gal, dec_gal

In [ ]:
if os.path.exists('../../Data/ps1_galaxy.csv') == False:

    table = ps1_casjobs(float(ra_gal), float(dec_gal), 0.3, maglim=20, quick = False)
    obj = table[(abs(table['raMean'] - float(ra_gal)) < 2/3600) & (abs(table['decMean'] - float(dec_gal)) < 2/3600)]
    obj.to_csv('../../Data/ps1_galaxy.csv', index=False)
else:
    obj = pd.read_csv('../../Data/ps1_galaxy.csv')

In [ ]:
# rgb = RGB([i_data, r_data, g_data], 
#           colours = ['red', 'green', 'blue'], intensities = [0.6, 1, 0.7], 
#           uppers = [99, 99, 99], lowers = [30, 30, 30], epsf = True,
#           save = False, save_name = 'test', save_folder = '/Users/zgl12/', 
#           gamma = 1.5, norm = 'linear', min_separation = 60, star_size = 15, 
#           epsf_plot=False, cross_corr = True)

rgb = RGB(colour_images, 
          save = False, save_name = 'sn2019vxm', save_folder = '/Users/zgl12/', 
          epsf_plot=False, epsf = True,
          bkg_plot = False, temp_save = True, run = False, manual_override=35)

colour_image = rgb.master_plot(colour_images, 
                               colours = ['blue', 'green', 'red'], 
                               intensities = [0.57, 1, 0.57], 
                               gamma = [1.5, 1.5, 1.5],
                               norms = ['linear', 'linear', 'linear',], 
                               uppers = [99, 99, 99,],
                               lowers = [30, 30, 30], 
                               interactive=False)



In [ ]:
angles = np.linspace(0, 2*np.pi, 1001) # 317/360 * 2 * np.pi

plt.figure(figsize=[fig_width, fig_width])
axs = plt.subplot(projection=wcs)

# axs = plt.subplot()

axs.imshow(colour_image, origin='lower', cmap='gray')

axs.plot([x[0], x_cen], [y[0], y_cen], c='lime', zorder = 0)
axs.scatter(x[0], y[0], c='yellow', s=350, marker='*', edgecolors='k', zorder = 10, label = r'SN$\,$2019vxm')
axs.scatter(x[1:], y[1:], c='r', s=150, zorder = 1, label = 'Control Lightcurves', marker = 'X', edgecolors='k', linewidths=0.75)


axs.grid(color='w', ls='dotted', alpha = 0.7)

axs.text(x_cen-35, y_cen - 46, f'Bright Object \n $m_{{r}}^{{}} = {obj["rMeanPSFMag"].values[0]:.1f}$', 
         color='yellow', fontsize = 16, fontweight='bold', 
         path_effects=[patheffects.withStroke(linewidth=0, foreground='k')])  # Adds edgecolor to the text)
axs.text(x_cen-28, y_cen + 27, f"$\delta$={coords['radius_arcsec'].iloc[0]:.1f}$''$", 
         color='lime', fontsize = 16, fontweight='bold', 
         path_effects=[patheffects.withStroke(linewidth=0, foreground='k')])  # Adds edgecolor to the text)

axs.set_xlim(x_cen-125, x_cen+125)
axs.set_ylim(y_cen-125, y_cen+125)

axs.legend()
# plt.savefig('../../Plots/atlas_control_loc.pdf', bbox_inches='tight', format='pdf')
plt.show()

In [ ]:
wcs